# 데이터셋 정의 (PyTorch)

In [4]:
import os
import json
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np


#  데이터셋 클래스

In [12]:
class SignLanguageDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.view_types = ["F", "L", "R", "D", "U"]
        self.data = []

        # 데이터 로드 디버깅
        if not os.path.exists(root_dir):
            print(f"❌ 오류: {root_dir} 경로가 존재하지 않습니다.")
            return
        
        print(f"📂 데이터셋 경로: {root_dir}")

        # 단어별 폴더 탐색
        for word_folder in sorted(os.listdir(root_dir)):
            word_path = os.path.join(root_dir, word_folder)
            if os.path.isdir(word_path):
                try:
                    # "WORD1501"에서 숫자 부분(1501)만 추출
                    word_label = int(word_folder.split("_")[2].replace("WORD", ""))
                except ValueError:
                    print(f"⚠️ Warning: 잘못된 폴더 형식 {word_folder}")
                    continue  # 잘못된 폴더 무시

                print(f"✅ 단어 폴더: {word_folder}, 라벨: {word_label}")

                # 뷰별 JSON 데이터 저장
                view_data = {view: [] for view in self.view_types}

                for view in self.view_types:
                    view_folder = os.path.join(word_path, f"{word_folder}_{view}")
                    if os.path.exists(view_folder):
                        json_files = sorted([f for f in os.listdir(view_folder) if f.endswith("_keypoints.json")])

                        if not json_files:
                            print(f"⚠️ {view_folder} 안에 JSON 파일 없음!")
                            continue

                        for json_file in json_files:
                            json_path = os.path.join(view_folder, json_file)
                            with open(json_path, "r") as f:
                                data = json.load(f)

                                if "people" in data and len(data["people"]) > 0:
                                    keypoints = []
                                    person = data["people"][0]

                                    # 얼굴, 포즈, 손 키포인트 병합
                                    for key in ["face_keypoints_2d", "pose_keypoints_2d", 
                                                "hand_left_keypoints_2d", "hand_right_keypoints_2d"]:
                                        if key in person:
                                            keypoints.extend(person[key])

                                    if keypoints:
                                        view_data[view].append(keypoints)

                # 모든 뷰 데이터가 있는 경우에만 추가
                if all(len(view_data[v]) > 0 for v in self.view_types):
                    print(f"✅ {word_folder} 데이터 추가됨")
                    self.data.append((view_data, word_label))
                else:
                    print(f"⚠️ {word_folder} 데이터 부족하여 추가되지 않음")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        view_data, label = self.data[idx]
        views = np.array([view_data[view] for view in self.view_types])  # (5, T, 42)
        views = torch.tensor(views, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        return views, label

# 데이터셋 및 DataLoader 생성
DATASET_PATH = "/home/usou/dev_ws/superbad/data/usou/New_sample/라벨링데이터/REAL/WORD/01_real_word_keypoint"
dataset = SignLanguageDataset(DATASET_PATH)

print(f"📊 데이터 개수: {len(dataset)}")

if len(dataset) > 0:
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
else:
    print("❌ 데이터셋이 비어 있어서 DataLoader를 생성할 수 없습니다.")


📂 데이터셋 경로: /home/usou/dev_ws/superbad/data/usou/New_sample/라벨링데이터/REAL/WORD/01_real_word_keypoint
✅ 단어 폴더: NIA_SL_WORD1501_REAL01_D, 라벨: 1501
⚠️ NIA_SL_WORD1501_REAL01_D 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1501_REAL01_F, 라벨: 1501
⚠️ NIA_SL_WORD1501_REAL01_F 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1501_REAL01_L, 라벨: 1501
⚠️ NIA_SL_WORD1501_REAL01_L 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1501_REAL01_R, 라벨: 1501
⚠️ NIA_SL_WORD1501_REAL01_R 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1501_REAL01_U, 라벨: 1501
⚠️ NIA_SL_WORD1501_REAL01_U 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1502_REAL01_D, 라벨: 1502
⚠️ NIA_SL_WORD1502_REAL01_D 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1502_REAL01_F, 라벨: 1502
⚠️ NIA_SL_WORD1502_REAL01_F 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1502_REAL01_L, 라벨: 1502
⚠️ NIA_SL_WORD1502_REAL01_L 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1502_REAL01_R, 라벨: 1502
⚠️ NIA_SL_WORD1502_REAL01_R 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA_SL_WORD1502_REAL01_U, 라벨: 1502
⚠️ NIA_SL_WORD1502_REAL01_U 데이터 부족하여 추가되지 않음
✅ 단어 폴더: NIA

In [13]:
print(f"데이터 개수: {len(dataset)}")


데이터 개수: 0


# 모델 정의

In [5]:
class MultiViewLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_views=5):
        super(MultiViewLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_views = num_views

        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size * num_views, num_classes)

    def forward(self, x):
        batch_size, num_views, seq_len, num_features = x.shape  # (B, 5, T, 42)

        view_outputs = []
        for i in range(num_views):
            out, _ = self.lstm(x[:, i, :, :])  # (B, T, H)
            view_outputs.append(out[:, -1, :])  # 마지막 타임스텝의 출력 사용

        combined = torch.cat(view_outputs, dim=-1)  # (B, H * 5)
        out = self.fc(combined)
        return out

# 모델 설정
input_size = 42  # 손 및 포즈 데이터 (21 x 2D 좌표)
hidden_size = 128
num_classes = 100  # 단어 개수에 맞게 조정

model = MultiViewLSTM(input_size, hidden_size, num_classes)


# 학습 코드

In [8]:
# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50  # 학습 횟수 조정 가능

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for views, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(views)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}, Accuracy: {correct/total:.4f}")

# 모델 저장
MODEL_PATH = "/mnt/data/sign_language_model.pth"
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")


NameError: name 'dataloader' is not defined